In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Uterine_Carcinosarcoma/GSE36133'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from the Cancer Cell Line Encyclopedia (CCLE)"
!Series_summary	"The Cancer Cell Line Encyclopedia (CCLE) project is a collaboration between the Broad Institute, the Novartis Institutes for Biomedical Research and the Genomics Novartis Foundation to conduct a detailed genetic and pharmacologic characterization of a large panel of human cancer models"
!Series_summary	"It consists of a compilation of gene expression, chromosomal copy number, and massively parallel sequencing data from nearly 1,000 human cancer cell lines. All raw and processed data are available through an integrated portal on www.broadinstitute.org/ccle"
!Series_overall_design	"The final cell line collection spans 36 cancer types. Representation of cell lines for each cancer type was mainly driven by cancer mortality in the United States, as a surrogate of unmet medical need, as well as availability."
Sample Characteristics Dictionary:
{0: ['primary site: central_ner

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Get the concatenated string of the complete background information
background_information = (
    "Expression data from the Cancer Cell Line Encyclopedia (CCLE). "
    "The Cancer Cell Line Encyclopedia (CCLE) project is a collaboration between the Broad Institute, the Novartis Institutes for Biomedical Research and the Genomics Novartis Foundation to conduct a detailed genetic and pharmacologic characterization of a large panel of human cancer models. "
    "It consists of a compilation of gene expression, chromosomal copy number, and massively parallel sequencing data from nearly 1,000 human cancer cell lines. All raw and processed data are available through an integrated portal on www.broadinstitute.org/ccle. "
    "The final cell line collection spans 36 cancer types. Representation of cell lines for each cancer type was mainly driven by cancer mortality in the United States, as a surrogate of unmet medical need, as well as availability."
)

# Step 2: Determine if gene expression data is available
if "gene expression" in background_information.lower():
    is_gene_available = True

# Step 3: Determine the availability of trait, age, and gender data based on sample characteristics obtained from the previous step
for key, values in sample_characteristics_dict.items():
    if any(isinstance(v, str) and 'age' in v.lower() for v in values):
        age_row = key
    if any(isinstance(v, str) and 'gender' in v.lower() for v in values):
        gender_row = key
    if (any(isinstance(v, str) and 'uterine_carcinosarcoma' in v.lower() for v in values) or 
        any(isinstance(v, str) and 'histology subtype' in v.lower() for v in values)):
        trait_row = key

# Step 4: Data Type Conversion Functions
def convert_trait(value):
    if isinstance(value, str) and "uterine_carcinosarcoma" in value.lower():
        return 1
    return 0

def convert_age(value):
    try:
        return float(value.split(':')[-1].strip())
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    if isinstance(value, str):
        if 'female' in value.lower():
            return 0
        elif 'male' in value.lower():
            return 1
    return None

# Step 5: Save Metadata
save_cohort_info('GSE36133', './preprocessed/Uterine_Carcinosarcoma/cohort_info.json', is_gene_available, trait_row is not None)

# Step 6: Clinical Feature Extraction 
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Uterine_Carcinosarcoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Uterine_Carcinosarcoma/trait_data/GSE36133.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM886835': [0], 'GSM886836': [0], 'GSM886837': [0], 'GSM886838': [0], 'GSM886839': [0], 'GSM886840': [0], 'GSM886841': [0], 'GSM886842': [0], 'GSM886843': [0], 'GSM886844': [0], 'GSM886845': [0], 'GSM886846': [0], 'GSM886847': [0], 'GSM886848': [0], 'GSM886849': [0], 'GSM886850': [0], 'GSM886851': [0], 'GSM886852': [0], 'GSM886853': [0], 'GSM886854': [0], 'GSM886855': [0], 'GSM886856': [0], 'GSM886857': [0], 'GSM886858': [0], 'GSM886859': [0], 'GSM886860': [0], 'GSM886861': [0], 'GSM886862': [0], 'GSM886863': [0], 'GSM886864': [0], 'GSM886865': [0], 'GSM886866': [0], 'GSM886867': [0], 'GSM886868': [0], 'GSM886869': [0], 'GSM886870': [0], 'GSM886871': [0], 'GSM886872': [0], 'GSM886873': [0], 'GSM886874': [0], 'GSM886875': [0], 'GSM886876': [0], 'GSM886877': [0], 'GSM886878': [0], 'GSM886879': [0], 'GSM886880': [0], 'GSM886881': [0], 'GSM886882': [0], 'GSM886883': [0], 'GSM886884': [0], 'GSM886885': [0], 'GSM886886': [0], 'GSM886887': [0], 'GSM886888': [0], 'GSM886889': [0], 'GSM88689

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['100009676_at', '10000_at', '10001_at', '10002_at', '10003_at',
       '100048912_at', '100049716_at', '10004_at', '10005_at', '10006_at',
       '10007_at', '10008_at', '100093630_at', '10009_at', '1000_at',
       '100101467_at', '100101938_at', '10010_at', '100113407_at', '10011_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'ORF': ['1', '10', '100', '1000', '10000'], 'Description': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'v-akt murine thymoma viral oncogene homolog 3 (protein kinase B, gamma)']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the necessary keys for mapping based on STEP3 and STEP5 outputs
identifier_key = 'ID'
gene_symbol_key = 'Description'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression data with mapped gene symbols
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Uterine_Carcinosarcoma/gene_data/GSE36133.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Uterine_Carcinosarcoma')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE36133', './preprocessed/Uterine_Carcinosarcoma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Uterine_Carcinosarcoma/GSE36133.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Uterine_Carcinosarcoma':
  25%: 0.0
  50% (Median): 0.0
  75%: 0.0
Min: 0.0
Max: 0.0
The distribution of the feature 'Uterine_Carcinosarcoma' in this dataset is severely biased.

A new JSON file was created at: ./preprocessed/Uterine_Carcinosarcoma/cohort_info.json
